# Downloading ERA5 Data

ERA5 data is downloaded using the NetCDF download protocols. Some variables may only be available in GRIB format - however, all standard climatological variables will be available using this script. ERA5 files are available in hourly timesteps; this code allows for resampling the data at lower frequencies (see the section on [Subsetting and Resampling](#subset)).

The ERA5 API must be installed first; follow this [Guide](https://cds.climate.copernicus.eu/api-how-to) for more information. (the easiest way is to use `pip` to install `cdsapi`).

ERA5 files are 0.25$^\circ$ hourly data - this results in very large files. This code is built on the assumption that for stability of the download and efficiency of memory use, data is downloaded one temporal chunk at a time (by default one year at a time). For reference, a 20 x 60 degree box by the equator of one variable takes around 10 minutes per year to prepare and download. Each temporal chunk is downloaded separately and saved in a temporary file (these files are useful for checking if the download is working correctly, and to start working with data) - once all the temporary files are downloaded, they are concatenated into one single file for the whole temporal range, and the temporary files are deleted. Set a bounding box for the download in `geographic_vars`.

Currently, only multiples of full years at a time can be downloaded; changing this is a relatively easy fix in the `c.retrieve()` call if it is needed.

The code checks for previously downloaded and processed files (based on the output temporal resolution and subset) and ignores them with a message (this is the case for both temporary and concatenated files - but previously downloaded temporary files are of course included in the concatenated file either way).

Files are placed in a subdirectory of the `raw_data_dir` set in the `dwnld_config.py` file `[raw_data_dir]/ERA5/`. If this subdirectory doesn't yet exist, it is created. Output files have the following filename: 

`[raw_data_dir]/ERA5/[short_name]_[output_freq_name]_ERA5_historical_reanalysis_[allyears[0]0101-allyears[-1]1231]_[fn_suffix].nc` 

with `[raw_data_dir]` set from the `config` file, `[short_name]` set in the `download_vars`, `output_freq_name` set in the `resampling_vars` (unless no time resampling is conducted; in this case `_hr_` is used), and `fn_suffix` set in the `geographic_vars`. 

**Summary of changes to downloaded ERA5 file**
- filename changed to CMIP5 standard, from the procedurally-generated unintelligible ERA5 file
- "latitude" changed to "lat" and "longitude" changed to "lon"
- variable short named changed to `download_vars['short_name']`
- data is resampled temporally (optional) from hourly to anything allowed by the `pandas` resampling syntax

**Useful links** 
Copernicus Data Store (where ERA5 data is located) data download page: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

ERA5 on the UCAR Climate Data Guide: https://climatedataguide.ucar.edu/climate-data/era5-atmospheric-reanalysis

ERA5 documentation: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation

# Setup

## Packages

In [ ]:
import cdsapi
import xarray as xr
import numpy as np
import datetime as dt
import os
from tqdm.notebook import tqdm
import re
%matplotlib inline

In [ ]:
# Get config file 
import dwnld_config as cfg

## Variables
Set which variables to download. 
- `short_name`: variable shorthand (e.g. `tas`) used as filename identifier (e.g. `tas_*.nc`) in the output filename and as the variable name in the file itself
- `long_name`: ERA5 variable identifier used to find file to download

The `long_name` for each variable can be found in the [ERA5 Parameter Database](https://apps.ecmwf.int/codes/grib/param-db) - for files available in NetCDF (filter using the NetCDF button on the left), the name will be listed as "cfName" in the table at the bottom of the page for each relevant variable, under the NetCDF tab.

In [ ]:
# Set which variables to download
download_vars = [{'short_name':'vwt','long_name':'vertical_integral_of_northward_water_vapour_flux'}, 
                  {'short_name':'uwt','long_name':'vertical_integral_of_eastward_water_vapour_flux'},
                  {'short_name':'u100','long_name':'100m_u_component_of_wind'},
                  {'short_name':'v100','long_name':'100m_v_component_of_wind'}]

## <a name="subset"></a>Subsetting and Resampling

In [ ]:
# Set geographic extent of download
geographic_vars = {'bbox':[13,33,-7,59], # Max lat, min lon, min lat, max lon
                   'fn_suffix':'_WIndOcean'}

In [ ]:
# Set years to process
all_years = np.arange(1979,2015)

# How many years to be processed at a time (may impact performance; detailed checks forthcoming)
chunk_size = 5

ERA5 data is released as hourly data. This code supports resampling using the `xarray.Dataset.resample()` function ([docs here](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html)), which is based on the `pandas.resample()` functionality. The next cell adjusts whether data is resampled. 

The `resample_level` value is piped into the `freq` call of the `.resample` indexer. Useful, common options include: 
- `D`: daily
- `M`: month (end)
- `MS`: month (beginning)

A full list of options is available in the `pandas` documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

In [ ]:
# Set resampling
resampling_vars = {'resamp':True, # whether to resample from hourly (if false, the rest is ignored)
                   'resample_level':'D', # what level to resample to 
                   'output_freq_name':'day'} # what the frequency should be called in the filename, attributes

# Downloading 

In [ ]:
if not os.path.exists(cfg.lpaths['raw_data_dir']):
    os.mkdir(cfg.lpaths['raw_data_dir'])
    print(cfg.lpaths['raw_data_dir']+' created')

In [ ]:
# Break up years into chunks
chunks = [all_years[i:i + chunk_size] for i in range(0, len(all_years), chunk_size)]

In [ ]:
c = cdsapi.Client()

In [ ]:
for download_var in download_vars:
    
    if resampling_vars['resamp']:
        fn_final = (cfg.lpaths['raw_data_dir']+'ERA5/'+download_var['short_name']+
                    '_'+resampling_vars['output_freq_name']+
                    '_ERA5_historical_reanalysis_'+
                    str(all_years[0])+'0101-'+str(all_years[-1])+'1231'+
                    geographic_vars['fn_suffix']+'.nc')
    else:
        fn_final = (cfg.lpaths['raw_data_dir']+'ERA5/'+download_var['short_name']+
                    '_hr_ERA5_historical_reanalysis_'+
                    str(all_years[0])+'0101-'+str(all_years[-1])+'1231'+
                    geographic_vars['fn_suffix']+'.nc')
    
    
    if not os.path.exists(fn_final):

        for chunk in chunks:
            # Set filename for temporary file
            fn0 = (cfg.lpaths['raw_data_dir']+'ERA5/'+download_var['short_name']+
                   '_hr_ERA5_historical_reanalysis_'+str(chunk[0])+'0101-'+str(chunk[-1])+'1231'+geographic_vars['fn_suffix']+'.nc')
            
            # If the file does not yet exist, download it
            if not os.path.exists(re.sub('hr','day',fn0)): 

                c.retrieve(
                    'reanalysis-era5-single-levels',
                    {
                        'product_type': 'reanalysis',
                        'format': 'netcdf',
                        'variable': download_var['long_name'],
                        'year': [str(ch) for ch in chunk],
                        'month': [
                            '01', '02', '03',
                            '04', '05', '06',
                            '07', '08', '09',
                            '10', '11', '12',
                        ],
                        'day': [
                            '01', '02', '03',
                            '04', '05', '06',
                            '07', '08', '09',
                            '10', '11', '12',
                            '13', '14', '15',
                            '16', '17', '18',
                            '19', '20', '21',
                            '22', '23', '24',
                            '25', '26', '27',
                            '28', '29', '30',
                            '31',
                        ],
                        'time': [
                            '00:00', '01:00', '02:00',
                            '03:00', '04:00', '05:00',
                            '06:00', '07:00', '08:00',
                            '09:00', '10:00', '11:00',
                            '12:00', '13:00', '14:00',
                            '15:00', '16:00', '17:00',
                            '18:00', '19:00', '20:00',
                            '21:00', '22:00', '23:00',
                        ],
                        'area': geographic_vars['bbox'],
                    },
                    fn0)


                tmp = xr.open_dataset(fn0)

                # Rename variables
                tmp = tmp.rename({'latitude':'lat','longitude':'lon'})

                if download_var['short_name'] not in tmp.var().variables.keys():
                    tmp = tmp.rename({[k for k in tmp.var().variables.keys()][0]:download_var['short_name']})

                if resampling_vars['resamp']:
                    # Resample temporally
                    tmp = tmp.resample(time=resampling_vars['resample_level']).mean()
                    fn1 = re.sub('hr',resampling_vars['output_freq_name'],fn0)
                    
                    # Export
                    tmp.to_netcdf(fn1)

                    # Remove old file
                    os.remove(fn0)
                else:
                    fn1 = fn0

                    # Remove old file (reverse order from the resampling case 
                    # above, which gives a new filename, because to_netcdf 
                    # doesn't overwrite)
                    os.remove(fn0)
                    
                    # Export
                    tmp.to_netcdf(fn1)


                print(fn1+' processed!')
            else: 
                print(fn1+' already exists; skipped.')

        # Open all the files (by wildcarding the date filename segment)
        ds = xr.open_mfdataset(cfg.lpaths['raw_data_dir']+'ERA5/'+download_var['short_name']+
                                '_'+resampling_vars['output_freq_name']+
                                '_ERA5_historical_reanalysis_*'+geographic_vars['fn_suffix']+'.nc',
                               combine='by_coords')

        # Save the concatenated file across all years 
        ds.to_netcdf(fn_final)

        # Remove the component files 
        for chunk in chunks:
            os.remove(cfg.lpaths['raw_data_dir']+'ERA5/'+download_var['short_name']+
                       '_'+resampling_vars['output_freq_name']+
                       '_ERA5_historical_reanalysis_'+
                       str(chunk[0])+'0101-'+str(chunk[-1])+'1231'+
                       geographic_vars['fn_suffix']+'.nc')
    else:
        print(fn_final+' already exists, skipped.')
        
        